# Audio Feature Extraction

## 1. Importing Library

In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
import librosa

## 2. Define function for extraction

In [2]:
def get_features(path, duration=4.0):

    signal, sr = librosa.load(path, sr=22050, duration=duration)

    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
    delta_mfcc = librosa.feature.delta(mfcc, mode='mirror')
    delta2_mfcc = librosa.feature.delta(mfcc, mode='mirror', order=2)

    mfccs = np.hstack((
                    np.mean(mfcc, axis=1),
                    np.std(mfcc, axis=1),
                    skew(mfcc, axis=1),
                    kurtosis(mfcc, axis=1)
                ))
    delta_mfccs = np.hstack((
                    np.mean(delta_mfcc, axis=1),
                    np.std(delta_mfcc, axis=1),
                    skew(delta_mfcc, axis=1),
                    kurtosis(delta_mfcc, axis=1)
                ))
    delta2_mfccs = np.hstack((
                    np.mean(delta2_mfcc, axis=1),
                    np.std(delta2_mfcc, axis=1),
                    skew(delta2_mfcc, axis=1),
                    kurtosis(delta2_mfcc, axis=1)
                ))

    extracted_features = np.hstack((mfccs, delta_mfccs, delta2_mfccs))
    return extracted_features

## 3. Get Audio Features for GPT Audio Data

In [3]:
dataset_path = '../data/guitar/wav'

data = []
for i, (root, _, files) in enumerate(os.walk(dataset_path)):
    if root is not dataset_path:
        if not files:
            tone_type = root.split('/')[-1]
        else:
            subtechnique = root.split('/')[-1]
            technique = subtechnique.split('_')[0]
            for file in files:
                file_path = os.path.join(root, file)
                features = get_features(file_path)
                features = np.append(features, [technique, subtechnique, tone_type, file_path])
                data.append(features)
print('Done!!!')

Done!!!


## 4. Define Columns Name to Create Dataframe

In [4]:
columns = []
names = ['mfcc', 'delta_mfcc', 'delta2_mfcc']
stats = ['mean', 'std', 'skew', 'kurtosis']
file_desc = ['technique', 'subtechnique', 'tone_type', 'file_path']

for name in names:
    for stat in stats:
        for i in range(13):
            col = f'{stat}_{name}_{i+1}'
            columns = np.append(columns, col)

columns = np.append(columns, file_desc)
columns.shape

(160,)

## 5. Convert Extracted Feature (List) to Dataframe

In [5]:
df = pd.DataFrame(data, columns=columns)
print(df.shape)

(6447, 160)


## 6. Save Dataframe to CSV File

In [6]:
df.to_csv('../data/gpt2.csv', index=False)

## 7. Load CSV File

In [76]:
gpt = pd.read_csv('../data/gpt.csv')
gpt

'../data/guitar/wav/5/trill/trill_3.wav'